In [4]:
!pip install tensorflow



In [7]:
!pip install music21


  Obtaining dependency information for music21 from https://files.pythonhosted.org/packages/a1/1b/ef7bdf01df19cc7ac9294531a9c991c6d382bde6bc15c9d106b9a5e547ef/music21-9.1.0-py3-none-any.whl.metadata
  Using cached music21-9.1.0-py3-none-any.whl.metadata (4.8 kB)
  Obtaining dependency information for jsonpickle from https://files.pythonhosted.org/packages/d6/be/c11f919e02622f15b942c0f2867dff6c3836ce32cd617627dda73bc9e5b7/jsonpickle-3.0.3-py3-none-any.whl.metadata
  Using cached jsonpickle-3.0.3-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for webcolors>=1.5 from https://files.pythonhosted.org/packages/d5/e1/3e9013159b4cbb71df9bd7611cbf90dc2c621c8aeeb677fc41dad72f2261/webcolors-1.13-py3-none-any.whl.metadata
  Using cached webcolors-1.13-py3-none-any.whl.metadata (2.6 kB)
Using cached music21-9.1.0-py3-none-any.whl (22.8 MB)
Using cached webcolors-1.13-py3-none-any.whl (14 kB)
Using cached jsonpickle-3.0.3-py3-none-any.whl (40 kB)
  Obtaining dependency informat

In [4]:
import tensorflow as tf
import numpy as np
from music21 import stream, note, chord, duration, midi, instrument


sequence_length = 100


n_features = 88  


model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(sequence_length, n_features), return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features, activation='softmax'))
])



# Music Generation
def generate_music(model, sequence_length, n_features, length=100, temperature=1.0, max_notes_to_print=10):
    start_sequence = np.random.rand(1, sequence_length, n_features)  # Random start sequence

    generated_notes = []

    # Generate notes/chords
    for i in range(length):
        prediction = model.predict(start_sequence)[0][-1]
        prediction = np.log(prediction) / temperature
        exp_preds = np.exp(prediction)
        normalized_preds = exp_preds / np.sum(exp_preds)
        index = np.random.choice(range(n_features), p=normalized_preds)

        generated_notes.append(index)

        # Print the generated note if it's within the limit
        if i < max_notes_to_print:
            print(f"Generated note {i+1}: {index}")

        # Update start sequence
        start_sequence[:, :-1, :] = start_sequence[:, 1:, :]
        start_sequence[:, -1, :] = np.eye(n_features)[index]

        # Break if the maximum number of notes to print is reached
        if i == max_notes_to_print - 1:
            print("Maximum number of notes to print reached. Stopping generation.")
            break

    return generated_notes

# Example usage:
generated_notes = generate_music(model, sequence_length, n_features, length=100, max_notes_to_print=10)
create_midi(generated_notes)



# Convert generated notes to MIDI
def create_midi(notes):
    output_notes = []
    offset = 0
    for pattern in notes:
        # Convert MIDI note index to string
        pattern_str = str(pattern)

        # Parse the pattern
        if '.' in pattern_str or pattern_str.isdigit():
            notes_in_chord = pattern_str.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern_str)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')




# Example usage:
generated_notes = generate_music(model, sequence_length, n_features, length=100)
create_midi(generated_notes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Generated note 1: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Generated note 2: 62
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Generated note 3: 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Generated note 4: 21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Generated note 5: 82
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Generated note 6: 85
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Generated note 7: 82
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Generated note 8: 12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Generated note 9: 63
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Generated note 10: 67
Maximum number of notes to print reached. Stopping generation.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Generated note 1: 78
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Generated note 2: 14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Generated note 3: 72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Generated note 4: 52
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Generated note 5: 54
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Generated note 